<a href="https://colab.research.google.com/github/jyryu3161/lecture/blob/main/Docking_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Molecular Docking 실습

# Installation

In [4]:
# download & extract
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
# move to python packages directory
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
# rdkit need libboost_python3.so.1.65.1
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4007    0  4007    0     0   3273      0 --:--:--  0:00:01 --:--:--  3276
100 20.2M  100 20.2M    0     0  3424k      0  0:00:06  0:00:06 --:--:-- 4490k


In [5]:
!git clone https://github.com/jyryu3161/lecture.git
!chmod u+x ./lecture/qvina2.1

Cloning into 'lecture'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [6]:
!pip install --pre deepchem
!pip install ogb
!pip install py3Dmol # 3D Molecular Visualizer
!pip install oddt
!pip install -U ProDy
!apt-get install pymol 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 4.2 MB/s 
     |████████████████████████████████| 22.7 MB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 3.4 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=e32536bfdec0e965de1a65609125da9fb7c42ea615dc8a544f46dbc6a890c4f5
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 4.3 MB/s 
     |████████████████████████████████| 51 kB 746 kB/s 
     |█████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.9 MB 3.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached biopython-1.76-cp37-cp37m-manylinux1_x86_64.whl (2.3 MB)
  Created wheel for ProDy: filename=ProDy-2.2.0-cp37-cp37m-linux_x86_64.whl size=10816240 sha256=5e92c294501a642c787dd18c42403a2ae7179efb109999bca7972423dd0f99d1
  Stored in directory: /root/.cache/pip/wheels/fc/59/47/97758bdbc4d5edf0aad1f7ccfeac5c687f6c2694d5efdb7571
Successfully built ProDy
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  apbs libglew2.0 libmaloc1 pymol-data python-opengl python-pmw
Sug

# Input setting

In [ ]:
pdb_id = '6M0K'
ligand_Id = 'FJC'
smiles = 'c1ccc2c(c1)cc([nH]2)C(=O)N[C@@H](Cc3cccc(c3)F)C(=O)N[C@@H](C[C@@H]4CCNC4=O)C=O'

docking_output_dir = './output/'

# Run molecular docking

In [ ]:
import py3Dmol
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

def make_3d_coordinate(m):
    target_mols = []
    m = Chem.AddHs(m)
    potential = AllChem.ETKDG()
    AllChem.EmbedMolecule(m, potential)
    cids = AllChem.EmbedMultipleConfs(m, numConfs=1, numThreads=15)
    
    for conf in m.GetConformers():
        tm = Chem.Mol(m,False,conf.GetId())
        res = AllChem.MMFFOptimizeMoleculeConfs(tm)
        target_mols.append(tm)
    return target_mols[0]

def drawit2(m,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p

def DrawComplex(protein,ligand):
    complex_pl = Chem.MolToPDBBlock(Chem.CombineMols(protein,ligand))
    view = py3Dmol.view(width=600,height=600)
    view.addModel(complex_pl,'pdb')
    chA = {'chain':['H','L','I']}
    chB = {'resn':'UNL'}
    view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
    #view.setStyle(chA,{'lines': {}})
    view.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)
    view.setStyle(chB,{'stick':{}})
    view.zoomTo()
    return view   


In [ ]:
from prody import *
import os

try:
    os.mkdir(docking_output_dir)
except:
    pass

pdb = parsePDB(pdb_id)

protein = pdb.select('protein')
ligand = pdb.select('resname %s'%(ligand_Id))

writePDB('protein.pdb', protein)
writePDB('ligand.pdb', ligand)


@> Connecting wwPDB FTP server RCSB PDB (USA).
DEBUG:.prody:Connecting wwPDB FTP server RCSB PDB (USA).
@> 6m0k downloaded (6m0k.pdb.gz)
DEBUG:.prody:6m0k downloaded (6m0k.pdb.gz)
@> PDB download via FTP completed (1 downloaded, 0 failed).
DEBUG:.prody:PDB download via FTP completed (1 downloaded, 0 failed).
@> 2560 atoms and 1 coordinate set(s) were parsed in 0.04s.
DEBUG:.prody:2560 atoms and 1 coordinate set(s) were parsed in 0.04s.


'ligand.pdb'

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

mol = Chem.MolFromSmiles(smiles)    
mol = Chem.AddHs(mol)

mol = make_3d_coordinate(mol)

with Chem.SDWriter('molecule.sdf') as w:
    w.write(mol)



In [ ]:
import oddt
from oddt.docking.AutodockVina import autodock_vina
from oddt.docking import AutodockVina

protein_file = './protein.pdb'
ligand_file = './ligand.pdb'

protein = next(oddt.toolkit.readfile('pdb', protein_file))
protein.protein = True
ligand = next(oddt.toolkit.readfile('pdb', ligand_file))

mol = Chem.MolFromMolFile('molecule.sdf')
mol2 = Chem.MolToMolBlock(mol)
dock_mol = oddt.toolkit.readstring('sdf', mol2)

protein.addh(only_polar=True)
ligand.addh(only_polar=True)

vina_obj = autodock_vina(protein=protein, auto_ligand=ligand, size=(20, 20, 20), num_modes=3, exhaustiveness=16, executable='./lecture/qvina2.1')

docking_outputs = vina_obj.dock([dock_mol])
docking_cnt = 1
for each_output in docking_outputs:
    vina_affinity = each_output.data['vina_affinity']
    AutodockVina.write_vina_pdbqt(each_output, docking_output_dir, name_id='docking_pose_%s'%(docking_cnt))
    
    print ("%s docking pose's binding energy (kcal/mol) : %s"%(docking_cnt, vina_affinity))
    docking_cnt+=1




1 docking pose's binding energy (kcal/mol) : -7.5
2 docking pose's binding energy (kcal/mol) : -7.4
3 docking pose's binding energy (kcal/mol) : -7.2


# Visualization

In [2]:
filename = './output/docking_pose_1.pdbqt'

ligand = next(oddt.toolkit.readfile('pdbqt', filename))

protein = Chem.MolFromPDBFile('protein.pdb')
DrawComplex(protein, ligand.Mol)


NameError: ignored